In [122]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [118]:
informations = []

i = 1
while len(informations) <= 10000:
    temp = []
    page = requests.get("https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag="+str(i))
    soup = BeautifulSoup(page.content, 'html.parser')
    features = soup.find_all('ul', attrs={'class': 'listing-features list-piped'})
    for annuncio in features:
        row = []
        for tag in annuncio:
            if tag != '\n':
                text = tag.get_text().strip().split()
                temp.append(text)
        if temp[0][0] != 'da' and len(temp) == 6 and temp[0][0] != 'SU':
            row.append(temp[0][1].replace('.','').replace('€',''))
            row.append(temp[1][0])
            row.append(temp[2][0])
            row.append(temp[3][0])
            row.append(temp[4][0])
            informations.append(row)   
        temp = []
    i+=1


KeyboardInterrupt: 

In [131]:
informations[1:10]

[['359000', '4', '130', '3', '3'],
 ['800000', '5+', '220', '3', '4'],
 ['950000', '5', '170', '1', '3'],
 ['250000', '4', '110', '2', 'A'],
 ['1090000', '5+', '190', '2', '2'],
 ['2150000', '4', '150', '1', '4'],
 ['2500000', '5+', '300', '3', '2'],
 ['650000', '5+', '193', '3', '4'],
 ['680000', '3', '90', '1', '3']]

In [124]:
d_informations = pd.DataFrame(informations)

In [135]:
d_informations.rename(columns={0 : "Prezzo", 1 : "Locali", 2 :"Superficie", 3 :"Bagni", 4 :"Piano"}).head(15)

,Prezzo,Locali,Superficie,Bagni,Piano
0,225000,2,50,1,1
1,359000,4,130,3,3
2,800000,5+,220,3,4
3,950000,5,170,1,3
4,250000,4,110,2,A
5,1090000,5+,190,2,2
6,2150000,4,150,1,4
7,2500000,5+,300,3,2
8,650000,5+,193,3,4
9,680000,3,90,1,3
